# Update attributes on reviewed parcels

In [1]:
import arcpy
from datetime import datetime
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# path output folder
outputs = '.\\Outputs'

In [3]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "wfrc_hui_20211203.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "wfrc_hui_20211203.gdb")
    
scratch = os.path.join(outputs, "scratch_rp.gdb")
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, "scratch_rp.gdb")

In [4]:
# load data
hui = r".\Current_Version\wfrc_hui_20211110.gdb\wfrc_hui_20211101"
hui_sdf = pd.DataFrame.spatial.from_featureclass(hui)
hui_sdf.shape

(391718, 22)

In [5]:
hui_sdf.columns

Index(['OBJECTID', 'PARCEL_ID', 'TYPE_WFRC', 'SUBTYPE_WFRC', 'NOTE', 'IS_OUG',
       'CITY', 'COUNTY', 'SUBCOUNTY', 'HOUSE_CNT', 'ADDR_CNT', 'UNIT_COUNT',
       'PARCEL_COUNT', 'DUA', 'FLOORS_CNT', 'PARCEL_ACRES', 'BLDG_SQFT',
       'TOTAL_MKT_VALUE', 'BUILT_YR', 'BUILT_DECADE', 'UNIT_ID', 'SHAPE'],
      dtype='object')

In [6]:
hui_sdf.head(5)

,OBJECTID,PARCEL_ID,TYPE_WFRC,SUBTYPE_WFRC,NOTE,IS_OUG,CITY,COUNTY,SUBCOUNTY,HOUSE_CNT,ADDR_CNT,UNIT_COUNT,PARCEL_COUNT,DUA,FLOORS_CNT,PARCEL_ACRES,BLDG_SQFT,TOTAL_MKT_VALUE,BUILT_YR,BUILT_DECADE,UNIT_ID,SHAPE
0,1,061900003,single_family,pud,['2 Story'],1,Woods Cross,DAVIS,South Davis County,18.0,18.0,18.0,20,9.339,2.0,1.927313,26873.0,4935167.0,2002.0,2000's,1,"{""rings"": [[[423773.07830000017, 4525135.77140..."
1,2,020950031,single_family,pud,[nan],1,Centerville,DAVIS,South Davis County,23.0,34.0,34.0,29,5.609,1.0,6.061556,47226.0,10518205.0,2007.0,2000's,2,"{""rings"": [[[425646.91280000005, 4531251.53360..."
2,3,022070001,single_family,pud,['2 Story'],1,Centerville,DAVIS,South Davis County,34.0,40.0,40.0,35,9.383,2.0,4.263039,52295.0,9844160.0,2007.0,2000's,3,"{""rings"": [[[425148.5184000004, 4531276.3881],..."
3,4,070700037,single_family,pud,[nan],1,Centerville,DAVIS,South Davis County,29.0,29.0,29.0,35,4.463,1.0,6.497216,55105.0,12752774.0,2016.0,2010's,4,"{""rings"": [[[425027.1202999996, 4533826.8158],..."
4,5,072580201,single_family,pud,['Ranch 1 Story'],1,Farmington,DAVIS,South Davis County,12.0,24.0,24.0,15,6.170,1.0,3.890075,19183.0,4691610.0,2011.0,2010's,5,"{""rings"": [[[425194.6507000001, 4535684.547900..."


In [7]:
# drop parcel id and parcel count
del hui_sdf['PARCEL_ID']
del hui_sdf['PARCEL_COUNT']

In [8]:
# add pud to note field and change to single family
hui_sdf.loc[(hui_sdf['SUBTYPE_WFRC'] == 'pud'), 'NOTE'] = hui_sdf['NOTE'] + ', PUD'
hui_sdf.loc[(hui_sdf['SUBTYPE_WFRC'] == 'pud'), 'SUBTYPE_WFRC'] = "single_family"
hui_sdf.loc[(hui_sdf['SUBTYPE_WFRC'] == 'mixed th/pud'), 'SUBTYPE_WFRC'] = "mixed th/single_family"
hui_sdf.head(5)

,OBJECTID,TYPE_WFRC,SUBTYPE_WFRC,NOTE,IS_OUG,CITY,COUNTY,SUBCOUNTY,HOUSE_CNT,ADDR_CNT,UNIT_COUNT,DUA,FLOORS_CNT,PARCEL_ACRES,BLDG_SQFT,TOTAL_MKT_VALUE,BUILT_YR,BUILT_DECADE,UNIT_ID,SHAPE
0,1,single_family,single_family,"['2 Story'], PUD",1,Woods Cross,DAVIS,South Davis County,18.0,18.0,18.0,9.339,2.0,1.927313,26873.0,4935167.0,2002.0,2000's,1,"{""rings"": [[[423773.07830000017, 4525135.77140..."
1,2,single_family,single_family,"[nan], PUD",1,Centerville,DAVIS,South Davis County,23.0,34.0,34.0,5.609,1.0,6.061556,47226.0,10518205.0,2007.0,2000's,2,"{""rings"": [[[425646.91280000005, 4531251.53360..."
2,3,single_family,single_family,"['2 Story'], PUD",1,Centerville,DAVIS,South Davis County,34.0,40.0,40.0,9.383,2.0,4.263039,52295.0,9844160.0,2007.0,2000's,3,"{""rings"": [[[425148.5184000004, 4531276.3881],..."
3,4,single_family,single_family,"[nan], PUD",1,Centerville,DAVIS,South Davis County,29.0,29.0,29.0,4.463,1.0,6.497216,55105.0,12752774.0,2016.0,2010's,4,"{""rings"": [[[425027.1202999996, 4533826.8158],..."
4,5,single_family,single_family,"['Ranch 1 Story'], PUD",1,Farmington,DAVIS,South Davis County,12.0,24.0,24.0,6.170,1.0,3.890075,19183.0,4691610.0,2011.0,2010's,5,"{""rings"": [[[425194.6507000001, 4535684.547900..."


In [9]:
hui_sdf_for_distribution = hui_sdf.copy()

In [10]:
# drop "wfrc" from type attributes
# rename parcel_acres to Acres
hui_sdf_for_distribution.rename(columns={'TYPE_WFRC':'TYPE',
                'SUBTYPE_WFRC':'SUBTYPE', 
                'PARCEL_ACRES':'ACRES',
                'BLDG_SQFT':'TOT_BD_FT2',
                'BUILT_YR':'APX_BLT_YR',
                'FLOORS_CNT':'APX_HGHT',
                'TOTAL_MKT_VALUE':'TOT_VALUE',
                'BUILT_DECADE':'BLT_DECADE'}, inplace=True)

In [11]:
hui_sdf.columns

Index(['OBJECTID', 'TYPE_WFRC', 'SUBTYPE_WFRC', 'NOTE', 'IS_OUG', 'CITY',
       'COUNTY', 'SUBCOUNTY', 'HOUSE_CNT', 'ADDR_CNT', 'UNIT_COUNT', 'DUA',
       'FLOORS_CNT', 'PARCEL_ACRES', 'BLDG_SQFT', 'TOTAL_MKT_VALUE',
       'BUILT_YR', 'BUILT_DECADE', 'UNIT_ID', 'SHAPE'],
      dtype='object')

In [12]:
# export to shape
hui_sdf.spatial.to_featureclass(location=os.path.join(gdb, 'wfrc_housing_unit_inventory_dev_20211203'),sanitize_columns=False)
hui_sdf_for_distribution.spatial.to_featureclass(location=os.path.join(gdb, 'wfrc_housing_unit_inventory_20211203'),sanitize_columns=False)

'E:\\Projects\\Housing-Unit-Inventory\\Outputs\\wfrc_hui_20211203.gdb\\wfrc_housing_unit_inventory_20211203'